# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-e565f07aca-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# GEO Workflow Series ("American Architects") 

Consider the following exploratory information need:

> You want to study the most prolific american architects

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P106`    | occupation    | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wd:Q5604`    | Frank Lloyd Wright     | node      |
| `wd:Q30`      | U.S.A.        | node |
| `wd:Q42973`   | architect     | node |
| `wd:Q2081276` | Pettit Memorial Chapel        | node |
| `wd:Q2977`    | Cathedral     | node |



Also consider

```
?p wdt:P27 wd:Q30  . 
?p wdt:P106 wd:Q42973  . 
```

is the BGP to retrieve all **american architects**

## Workload Goals

1. Identify the BGP to obtain the notable works of an architect and their location

2. Identify the BGP that connect a building with its architect, type, and architectural style

3. Which architect is the most prolific in the U.S. and outside?

4. Analyze the location of the buildings designed by the american architects
 
   4.1 Which styles exists in the U.S.? Which type of building?
   
   4.2 In which U.S. states there are more notable buildings?
   
   4.3 Which U.S. state contains the largest number of building designed by U.S. architects


#### Example of query

In [2]:
queryString = """
SELECT COUNT(*) WHERE { 

    ?p wdt:P27 wd:Q30  . 
    ?p wdt:P106 wd:Q42973  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '3354')]


1

### My Workflow

#### ***Task 1 :*** Identify the BGP to obtain the notable works of an architect and their location.

First of all, I need to understand how an ***architect (wd:Q42973)*** is related to its works. 

To do this, I show all the properties between istances of ***architect (wd:Q42973)*** and the ***Pettit Memorial Chapel (wd:Q2081276)***.

In [3]:
queryString = """
SELECT ?architectName ?p ?pName WHERE { 

    # Retrieve all the architects and connect them to Pettit Memorial Chapel
    ?architect  wdt:P106 wd:Q42973   ;
                ?p       wd:Q2081276 .

    # This returns the labels
    ?architect <http://schema.org/name> ?architectName .
    ?p         <http://schema.org/name> ?pName         .
} 
"""

print("Results")
run_query(queryString)

Results
[('architectName', 'Frank Lloyd Wright'), ('p', 'http://www.wikidata.org/prop/direct/P800'), ('pName', 'notable work')]


1

I discovered the property ***notable work (wdt:P800)***. I use it on other istances of ***architect (wd:Q42973)*** to show their notable works.

In [4]:
queryString = """
SELECT ?architectName ?notableWorkName WHERE { 

    # Retrieve all the architects and their notable works
    ?architect  wdt:P106  wd:Q42973    ;
                wdt:P800  ?notableWork .

    # This returns the labels
    ?architect   <http://schema.org/name> ?architectName   .
    ?notableWork <http://schema.org/name> ?notableWorkName .
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('architectName', 'Amós Salvador Sáenz y Carreras'), ('notableWorkName', 'Dispensario antituberculoso Victoria Eugenia')]
[('architectName', 'Amós Salvador Sáenz y Carreras'), ('notableWorkName', 'Gal Factory')]
[('architectName', 'Antonio Lamela'), ('notableWorkName', 'Estadio Las Gaunas')]
[('architectName', 'Antonio Lamela'), ('notableWorkName', 'Torre Caja Badajoz')]
[('architectName', 'Antonio Lamela'), ('notableWorkName', 'Edificio Pirámide, Madrid')]
[('architectName', 'Antonio Lamela'), ('notableWorkName', 'Edificio Galaxia, Madrid')]
[('architectName', 'Antonio Lamela'), ('notableWorkName', 'Edificio de viviendas en Paseo de la Castellana 121-123, Madrid')]
[('architectName', 'Antonio Lamela'), ('notableWorkName', 'Torres de Colón')]
[('architectName', 'Antonio Lamela'), ('notableWorkName', 'Motel El Hidalgo')]
[('architectName', 'Antonio Lamela'), ('notableWorkName', 'Ciudad Real Madrid')]


10

It works as expected. Now I have to understand how notable works are related to a specific location.

I show all the properties of the ***Pettit Memorial Chapel (wd:Q2081276)***.

In [5]:
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 

    # Retrieve everything connected to Pettit Memorial Chapel
    wd:Q2081276 ?p ?o .

    # This returns the labels
    ?p <http://schema.org/name> ?pName .    
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1435'), ('pName', 'heritage designation')]
[('p', 'http://www.wikidata.org/prop/direct/P149'), ('pName', 'architectural style')]
[('p', 'http://www.wikidata.org/prop/direct/P1566'), ('pName', 'GeoNames ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P5383'), ('pName', 'archINFORM project ID')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P84'), ('pName', 'architect')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main ca

14

There are two property that seems related to location:
* ***located in the administrative territorial entity (wdt:P131)***
* ***country (wdt:P17)***

I try to see what is connected through these two properties.

In [6]:
queryString = """
SELECT DISTINCT ?o ?oName WHERE { 

    # Retrieve everything connect to Pettit Memorial Chapel through property Country
    wd:Q2081276 wdt:P17 ?o .

    # This returns the labels
    ?o <http://schema.org/name> ?oName .    
}
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]


1

The property ***country (wdt:P17)*** returns (as expected) the country where the notable work is located. 

I try to use ***located in the administrative territorial entity (wdt:P131)***.

In [7]:
queryString = """
SELECT DISTINCT ?o ?oName WHERE { 

    # Retrieve everything connect to Pettit Memorial Chapel through property located in the administrative territorial entity
    wd:Q2081276 wdt:P131 ?o .

    # This returns the labels
    ?o <http://schema.org/name> ?oName .    
}
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q185609'), ('oName', 'Belvidere')]


1

I don't know what ***Belvidere (wd:Q185609)*** is, so I use the property ***instanceOf (wdt:P31)***.

In [8]:
queryString = """
SELECT DISTINCT ?class ?className WHERE { 

    # Retrieve class of Belvidere
    wd:Q185609 wdt:P31 ?class .

    # This returns the labels
    ?class <http://schema.org/name> ?className .    
}
"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q1093829'), ('className', 'city of the United States')]
[('class', 'http://www.wikidata.org/entity/Q62049'), ('className', 'county seat')]


2

***Belvidere (wd:Q185609)*** is actually a city in the United States. So, also the property ***located in the administrative territorial entity (wdt:P131)*** returns useful information about the location of notable works.

In my opinion the BGP to retrieve notable works of an architect and their location should consider both of them:

```
?architect  wdt:P106  wd:Q42973 ;
            wdt:P800  ?work     .           
?work       wdt:P17   ?country  ;
            wdt:P131  ?city     .
```

I want to try it with a specific architect such as ***Frank Lloyd Wright (wd:Q5604)***.

In [9]:
queryString = """
SELECT ?notableWorkName ?countryName ?locationName WHERE { 

    # Retrieve all notable works of Frank Lloyd Wright
    wd:Q5604  wdt:P800  ?notableWork .
    
    # Retrieve the countries and the cities where the notable works are located
    ?notableWork   wdt:P17  ?country  ;
                   wdt:P131 ?location .
            
    # This returns the labels
    ?notableWork <http://schema.org/name> ?notableWorkName .
    ?country     <http://schema.org/name> ?countryName     .
    ?location    <http://schema.org/name> ?locationName    .
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('notableWorkName', 'Pettit Memorial Chapel'), ('countryName', 'United States of America'), ('locationName', 'Belvidere')]
[('notableWorkName', 'Solomon R. Guggenheim Museum'), ('countryName', 'United States of America'), ('locationName', 'Manhattan')]
[('notableWorkName', 'Fallingwater'), ('countryName', 'United States of America'), ('locationName', 'Stewart Township')]
[('notableWorkName', 'A. D. German Warehouse'), ('countryName', 'United States of America'), ('locationName', 'Richland Center')]
[('notableWorkName', 'Annunciation Greek Orthodox Church'), ('countryName', 'United States of America'), ('locationName', 'Wauwatosa')]
[('notableWorkName', 'Emil Bach House'), ('countryName', 'United States of America'), ('locationName', 'Chicago')]


6

#### END TASK 1:
The property ***notable work (wdt:P800)*** can be used to retrieve notable works of architects.

Both ***located in the administrative territorial entity (wdt:P131)*** and ***country (wdt:P17)*** gives useful information about the location of a notable work.

To conclude the BGP to obtain the notable works of an architect and their location should be:
```
?architect  wdt:P106  wd:Q42973 ;
            wdt:P800  ?work     .           
?work       wdt:P17   ?country  ;
            wdt:P131  ?city     .
```

### ***Task 2 :*** Identify the BGP that connect a building with its architect, type, and architectural style.

I show again all the properties of the ***Pettit Memorial Chapel (wd:Q2081276)***.

In [10]:
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 

    # Retrieve everything connect to Pettit Memorial Chapel
    wd:Q2081276 ?p ?o .

    # This returns the labels
    ?p <http://schema.org/name> ?pName .    
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1435'), ('pName', 'heritage designation')]
[('p', 'http://www.wikidata.org/prop/direct/P149'), ('pName', 'architectural style')]
[('p', 'http://www.wikidata.org/prop/direct/P1566'), ('pName', 'GeoNames ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P5383'), ('pName', 'archINFORM project ID')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P84'), ('pName', 'architect')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main ca

14

I notice that there is a property called ***architect(wdt:P84)***. 

I want to better clarify the distiction between ***architect(wdt:P84)*** and ***notable work (wdt:P800)***.

I use ***architect(wdt:P84)*** on ***Pettit Memorial Chapel (wd:Q2081276)***

In [11]:
queryString = """
SELECT DISTINCT ?architect ?architectName WHERE { 

    # Retrieve everything connect to Pettit Memorial Chapel
    wd:Q2081276 wdt:P84 ?architect .

    # This returns the labels
    ?architect <http://schema.org/name> ?architectName .    
}
"""

print("Results")
run_query(queryString)

Results
[('architect', 'http://www.wikidata.org/entity/Q5604'), ('architectName', 'Frank Lloyd Wright')]


1

For a single notable work I obtain the same result as using ***notable work (wdt:P800)***. 

Now I try to retrieve all the works of ***Frank Lloyd Wright (wd:Q5604)*** by using ***architect(wdt:P84)***.

In [12]:
queryString = """
SELECT DISTINCT ?work ?workName WHERE { 

    # Retrieve works of Frank Lloyd Wright
    ?work wdt:P84 wd:Q5604 .

    # This returns the labels
    ?work <http://schema.org/name> ?workName .    
}
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('work', 'http://www.wikidata.org/entity/Q929965'), ('workName', 'Robie House')]
[('work', 'http://www.wikidata.org/entity/Q2081276'), ('workName', 'Pettit Memorial Chapel')]
[('work', 'http://www.wikidata.org/entity/Q965011'), ('workName', 'Millard House')]
[('work', 'http://www.wikidata.org/entity/Q201469'), ('workName', 'Solomon R. Guggenheim Museum')]
[('work', 'http://www.wikidata.org/entity/Q2424445'), ('workName', 'Thomas H. Gale House')]
[('work', 'http://www.wikidata.org/entity/Q4956682'), ('workName', 'Brandes House')]
[('work', 'http://www.wikidata.org/entity/Q5468984'), ('workName', 'Forest House')]
[('work', 'http://www.wikidata.org/entity/Q6531634'), ('workName', 'Robert M. Lamp House')]
[('work', 'http://www.wikidata.org/entity/Q7010799'), ('workName', 'New Orleans Union Station')]
[('work', 'http://www.wikidata.org/entity/Q64683561'), ('workName', 'Sherman M. Booth House')]
[('work', 'http://www.wikidata.org/entity/Q7296752'), ('workName', 'Ravine Bluffs Develo

50

So, I have confirmed that ***notable work (wdt:P800)*** returns only the notable works of an architect, while ***architect(wdt:P84)*** can be used for every work of a specific architect.

For the architectural style, I can use the property ***architectural style(wdt:P149)*** discovered in a previous query. 

I try it on the notable works of ***Frank Lloyd Wright (wd:Q5604)***.


In [13]:
queryString = """
SELECT ?notableWorkName ?styleName WHERE { 

    # Retrieve all notable works of Frank Lloyd Wright
    wd:Q5604  wdt:P800  ?notableWork .
    
    # Retrieve the style of the notable works
    ?notableWork wdt:P149 ?style .
            
    # This returns the labels
    ?notableWork <http://schema.org/name> ?notableWorkName .
    ?style <http://schema.org/name> ?styleName .
} 
"""

print("Results")
run_query(queryString)

Results
[('notableWorkName', 'Solomon R. Guggenheim Museum'), ('styleName', 'modern architecture')]
[('notableWorkName', 'Pettit Memorial Chapel'), ('styleName', 'modern architecture')]
[('notableWorkName', 'Fallingwater'), ('styleName', 'modern architecture')]
[('notableWorkName', 'Emil Bach House'), ('styleName', 'Prairie School architecture')]
[('notableWorkName', 'A. D. German Warehouse'), ('styleName', 'Mayan Revival architecture')]


5

There is no a specific property for the type of a building, so I think I should use the property ***instanceOf (wdt:P31)***. 

I try it on ***Pettit Memorial Chapel (wd:Q2081276)***.

In [14]:
queryString = """
SELECT DISTINCT ?class ?className WHERE { 

    # Retrieve class of Pettit Memorial Chapel
    wd:Q2081276 wdt:P31 ?class .
            
    # This returns the labels
    ?class <http://schema.org/name> ?className .
} 
"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q108325'), ('className', 'chapel')]


1

I can try to use ***subclassOf (wdt:P279)*** on ***chapel (wd:Q108325)***.

In [15]:
queryString = """
SELECT DISTINCT ?class ?className WHERE { 

    # Use subclassOf on Chapel
    wd:Q108325 wdt:P279 ?class .
            
    # This returns the labels
    ?class <http://schema.org/name> ?className .
} 
"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q16970'), ('className', 'church building')]
[('class', 'http://www.wikidata.org/entity/Q24398318'), ('className', 'religious building')]


2

I try to use ***subclassOf (wdt:P279)*** on ***religious building (wd:Q24398318)***.

In [16]:
queryString = """
SELECT DISTINCT ?class ?className WHERE { 

    # Use subclass of on religious building
    wd:Q24398318 wdt:P279 ?class .
            
    # This returns the labels
    ?class <http://schema.org/name> ?className .
} 
"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q21029893'), ('className', 'religious object')]
[('class', 'http://www.wikidata.org/entity/Q41176'), ('className', 'building')]
[('class', 'http://www.wikidata.org/entity/Q1370598'), ('className', 'place of worship')]


3

By using ***building (wd:Q41176)***, I can retrieve all the buildings using this BGP:
```
?building wdt:P31/wdt:279* wd:Q41176 .
```

In [17]:
queryString = """
SELECT DISTINCT ?building ?buildingName WHERE { 

    # Retrieve buildings
    ?building wdt:P31/wdt:279* wd:Q41176 .
            
    # This returns the labels
    ?building <http://schema.org/name> ?buildingName .
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('building', 'http://www.wikidata.org/entity/Q15885886'), ('buildingName', 'Torre Caja Badajoz')]
[('building', 'http://www.wikidata.org/entity/Q24439272'), ('buildingName', 'Edificio Pirámide, Madrid')]
[('building', 'http://www.wikidata.org/entity/Q26834201'), ('buildingName', 'Edificio Galaxia, Madrid')]
[('building', 'http://www.wikidata.org/entity/Q27032920'), ('buildingName', 'Edificio de viviendas en Paseo de la Castellana 121-123, Madrid')]
[('building', 'http://www.wikidata.org/entity/Q27243286'), ('buildingName', 'Dispensario antituberculoso Victoria Eugenia')]
[('building', 'http://www.wikidata.org/entity/Q3471746'), ('buildingName', 'Sanctuaire of San Migel de Aralar, Uharte-Arakil')]
[('building', 'http://www.wikidata.org/entity/Q46229138'), ('buildingName', "Edifici del Casino l'Aliança del Poblenou")]
[('building', 'http://www.wikidata.org/entity/Q8196155'), ('buildingName', 'Almudín de Játiva')]
[('building', 'http://www.wikidata.org/entity/Q8350472'), ('buildi

20

I can finally connect a ***building (wd:Q41176)*** with its architect, type, and architectural style.

I use ***GROUP_CONCAT*** because I can have multilpe architects,styles and types.

In [18]:
queryString = """
SELECT DISTINCT ?buildingName (GROUP_CONCAT(DISTINCT ?architectName ; separator = ", ") AS ?architects) (GROUP_CONCAT(DISTINCT ?styleName ; separator = ", ") AS ?styles)
                              (GROUP_CONCAT(DISTINCT ?typeName ; separator = ", ") AS ?types) WHERE { 

    # Retrieve buildings, their architects, their styles and types.
    ?building   wdt:P31/wdt:279* wd:Q41176 ;
                wdt:P84  ?architect ;
                wdt:P149 ?style     ;           
                wdt:P31  ?type      .

    # I'm only interested in classes differant from BUILDING
    FILTER (?type != wd:Q41176)            
            
    # This returns the labels
    ?building <http://schema.org/name> ?buildingName .
    ?architect <http://schema.org/name> ?architectName .
    ?style <http://schema.org/name> ?styleName .
    ?type <http://schema.org/name> ?typeName .
    
}
GROUP BY ?buildingName
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('buildingName', 'Tamminiemi'), ('architects', 'Gustaf Strengell, Sigurd Frosterus'), ('styles', 'Art Nouveau'), ('types', 'government building, historic house museum, main house, museum, residential building, villa')]
[('buildingName', 'Casa Llosas'), ('architects', 'Adolf Ruiz i Casamitjana'), ('styles', 'modernism'), ('types', 'house')]
[('buildingName', 'Eduskuntatalo'), ('architects', 'J. S. Sirén'), ('styles', 'Nordic Classicism'), ('types', 'administrative building, capitol building')]
[('buildingName', 'Casa Gasull'), ('architects', 'Lluís Domènech i Montaner'), ('styles', 'Art Nouveau, modernism'), ('types', 'masia')]
[('buildingName', 'Deutsche Hypothekenbank Meiningen'), ('architects', 'Eduard Fritze'), ('styles', 'eclectic architecture'), ('types', 'large bank, mortgage bank')]
[('buildingName', 'Category:Wedding Palace in Vyborg'), ('architects', 'Brynolf Blomqvist'), ('styles', 'Eclectic architecture in Russia, Renaissance Revival architecture'), ('types', 'landm

20

#### END TASK 2:
I can finally answer to the initial question. The BGP that connect a building with its architect, type, and architectural style is:
```
?building   wdt:P31/wdt:279* wd:Q41176  ;
            wdt:P84          ?architect ;
            wdt:P149         ?style     ;           
            wdt:P31          ?type      .
```

I have to add ***FILTER (?type != wd:Q41176)*** because in same case the property ***instanceOf (wdt:P31)*** returns directly ***building (wd:Q41176)*** instead of a type of ***building (wd:Q41176)***.

### ***Task 3 :*** Which architect is the most prolific in the U.S. and outside?

To retrieve the most prolific architect, I can count how many buildings are related to each architect (of course other criteria can be considered).

To answer this question, I can rely on different URI retrieved before:
* ***building (wd:Q41176)***
* ***country (wdt:P17)***
* ***architect(wdt:P84)***
* ***United States of America (wd:Q30)***

I start counting how many building in the ***United States of America (wd:Q30)*** are related to each architect.

In [19]:
queryString = """
SELECT ?architect ?architectName COUNT(DISTINCT ?building) AS ?numBuildings WHERE { 

    # Retrieve buildings in U.S and their architects
    ?building   wdt:P31/wdt:279* wd:Q41176  ;
                wdt:P17          wd:Q30     ;
                wdt:P84          ?architect .
    
    # Retrieve all the architects 
    ?architect  wdt:P106 wd:Q42973 .

    # This returns the labels
    ?architect <http://schema.org/name> ?architectName .
}
GROUP BY ?architect ?architectName
ORDER BY DESC(?numBuildings)
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('architect', 'http://www.wikidata.org/entity/Q6137444'), ('architectName', 'James Knox Taylor'), ('numBuildings', '10')]
[('architect', 'http://www.wikidata.org/entity/Q5604'), ('architectName', 'Frank Lloyd Wright'), ('numBuildings', '8')]
[('architect', 'http://www.wikidata.org/entity/Q5497817'), ('architectName', 'Frederick G. Clausen'), ('numBuildings', '7')]
[('architect', 'http://www.wikidata.org/entity/Q706666'), ('architectName', 'Albert Kahn'), ('numBuildings', '7')]
[('architect', 'http://www.wikidata.org/entity/Q6128314'), ('architectName', 'James A. Wetmore'), ('numBuildings', '7')]
[('architect', 'http://www.wikidata.org/entity/Q7411623'), ('architectName', 'Samuel Hannaford'), ('numBuildings', '6')]
[('architect', 'http://www.wikidata.org/entity/Q464603'), ('architectName', 'Cass Gilbert'), ('numBuildings', '6')]
[('architect', 'http://www.wikidata.org/entity/Q8009244'), ('architectName', 'William Foster'), ('numBuildings', '5')]
[('architect', 'http://www.wikid

10

Now, I try with buildings outside ***United States of America (wd:Q30)***.

In [20]:
queryString = """
SELECT ?architect ?architectName COUNT(DISTINCT ?building) AS ?numBuildings WHERE { 

    # Retrieve buildings,their countries and their architects
    ?building   wdt:P31/wdt:279* wd:Q41176  ;
                wdt:P17          ?country   ;
                wdt:P84          ?architect .
    
    # Retrieve all the architects 
    ?architect  wdt:P106 wd:Q42973 .
    
    # I want to exclude buildings in the U.S
    FILTER (?country != wd:Q30)

    # This returns the labels
    ?architect <http://schema.org/name> ?architectName .
}
GROUP BY ?architect ?architectName
ORDER BY DESC(?numBuildings)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('architect', 'http://www.wikidata.org/entity/Q9028072'), ('architectName', 'Manuel Joaquim Raspall i Mayol'), ('numBuildings', '54')]
[('architect', 'http://www.wikidata.org/entity/Q9095271'), ('architectName', 'Víctor Eusa'), ('numBuildings', '44')]
[('architect', 'http://www.wikidata.org/entity/Q9023348'), ('architectName', 'Lluís Muncunill i Parellada'), ('numBuildings', '43')]
[('architect', 'http://www.wikidata.org/entity/Q1150065'), ('architectName', 'Cèsar Martinell i Brunet'), ('numBuildings', '36')]
[('architect', 'http://www.wikidata.org/entity/Q7167730'), ('architectName', 'Pere Caselles i Tarrats'), ('numBuildings', '34')]
[('architect', 'http://www.wikidata.org/entity/Q212465'), ('architectName', 'Enrique Nieto'), ('numBuildings', '33')]
[('architect', 'http://www.wikidata.org/entity/Q27075131'), ('architectName', 'Else Aropaltio'), ('numBuildings', '30')]
[('architect', 'http://www.wikidata.org/entity/Q77103'), ('architectName', 'Carl Ludvig Engel'), ('numBuildi

10

I want to try a different criterion. 

Instead of counting all the buildings related to an architect, I count only his notable works using the property ***notable work (wdt:P800)***.

In [21]:
queryString = """
SELECT ?architect ?architectName COUNT(DISTINCT ?building) AS ?numBuildings WHERE { 

    # I retrieve only the notable works of architects
    ?architect  wdt:P106 wd:Q42973 ;
                wdt:P800 ?building .

    # Retrieve buildings in U.S
    ?building   wdt:P31/wdt:279* wd:Q41176 ;
                wdt:P17          wd:Q30    .
                      
    # This returns the labels
    ?architect <http://schema.org/name> ?architectName .
}
GROUP BY ?architect ?architectName
ORDER BY DESC(?numBuildings)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('architect', 'http://www.wikidata.org/entity/Q15996171'), ('architectName', 'Egerton Swartwout'), ('numBuildings', '4')]
[('architect', 'http://www.wikidata.org/entity/Q2926468'), ('architectName', 'Bruce Price'), ('numBuildings', '2')]
[('architect', 'http://www.wikidata.org/entity/Q4889909'), ('architectName', 'Benno Janssen'), ('numBuildings', '2')]
[('architect', 'http://www.wikidata.org/entity/Q16992139'), ('architectName', 'Henry H. Dupont'), ('numBuildings', '2')]
[('architect', 'http://www.wikidata.org/entity/Q1277977'), ('architectName', 'Paul Rudolph'), ('numBuildings', '2')]
[('architect', 'http://www.wikidata.org/entity/Q7381855'), ('architectName', 'Russell Warren'), ('numBuildings', '1')]
[('architect', 'http://www.wikidata.org/entity/Q102184859'), ('architectName', 'George S. Kingsley'), ('numBuildings', '1')]
[('architect', 'http://www.wikidata.org/entity/Q15485556'), ('architectName', 'Wyatt C. Hedrick'), ('numBuildings', '1')]
[('architect', 'http://www.wiki

10

It is possible to notice that the results are different from the query that count all the buildings. Now I repeat the query with buildings outside ***United States of America (wd:Q30)***.

In [22]:
queryString = """
SELECT ?architect ?architectName COUNT(DISTINCT ?building) AS ?numBuildings WHERE { 

    # I retrieve only the notable works of architects
    ?architect  wdt:P106 wd:Q42973 ;
                wdt:P800 ?building .

    # Retrieve buildings and their countries
    ?building   wdt:P31/wdt:279* wd:Q41176 ;
                wdt:P17          ?country  .
                
    # I want to exclude buildings in the U.S
     FILTER (?country != wd:Q30)
            
    # This returns the labels
    ?architect <http://schema.org/name> ?architectName .
}
GROUP BY ?architect ?architectName
ORDER BY DESC(?numBuildings)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('architect', 'http://www.wikidata.org/entity/Q3324260'), ('architectName', 'Luis Gutiérrez Soto'), ('numBuildings', '12')]
[('architect', 'http://www.wikidata.org/entity/Q6098119'), ('architectName', 'Rafael de La-Hoz Castanys'), ('numBuildings', '10')]
[('architect', 'http://www.wikidata.org/entity/Q5946144'), ('architectName', 'José Yarnoz Larrosa'), ('numBuildings', '9')]
[('architect', 'http://www.wikidata.org/entity/Q28976579'), ('architectName', 'José de Astiz'), ('numBuildings', '7')]
[('architect', 'http://www.wikidata.org/entity/Q212465'), ('architectName', 'Enrique Nieto'), ('numBuildings', '6')]
[('architect', 'http://www.wikidata.org/entity/Q20031628'), ('architectName', 'Eleuterio Población Knappe'), ('numBuildings', '6')]
[('architect', 'http://www.wikidata.org/entity/Q2857931'), ('architectName', 'Aníbal González'), ('numBuildings', '6')]
[('architect', 'http://www.wikidata.org/entity/Q3960878'), ('architectName', 'Silvio Gambini'), ('numBuildings', '5')]
[('ar

10

Also for buildings outside buildings outside ***United States of America (wd:Q30)***, it is possible to notice that the results are different from the query that use the property ***architect(wdt:P84)***.

#### END TASK 3:
It is possible to determine different criteria to find the most prolific architect in the U.S. and outside. I used two different criteria.

First: I count how many buildings are related to each architect using the property ***architect(wdt:P84)***.
* Inside U.S. : ***James Knox Taylor*** with 10 buildings
* Outside U.S : ***Manuel Joaquim Raspall i Mayol*** with 54 buildings

Second: I count only the notable works using the property ***notable work (wdt:P800)***.
* Inside U.S. : ***Egerton Swartwout*** with 4 notable works
* Outside U.S : ***Luis Gutiérrez Soto*** with 12 notable works

### ***Task 4.1 :*** Which styles exists in the U.S.? Which type of building?

I show the most used styles in the U.S buildings relyng on the property ***architectural style(wdt:P149)*** discovered in a previous query.

In [23]:
queryString = """
SELECT ?style ?styleName COUNT(DISTINCT ?building) AS ?numBuildings WHERE { 
    
    # Retrieve buildings in U.S and their style
    ?building   wdt:P31/wdt:279* wd:Q41176 ;
                wdt:P17          wd:Q30 ;
                wdt:P149         ?style .
                
    # This returns the labels
    ?style <http://schema.org/name> ?styleName .
}
GROUP BY ?style ?styleName
ORDER BY DESC(?numBuildings)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('style', 'http://www.wikidata.org/entity/Q1513688'), ('styleName', 'Greek Revival architecture'), ('numBuildings', '523')]
[('style', 'http://www.wikidata.org/entity/Q502163'), ('styleName', 'Renaissance Revival architecture'), ('numBuildings', '378')]
[('style', 'http://www.wikidata.org/entity/Q54111'), ('styleName', 'Neoclassical architecture'), ('numBuildings', '157')]
[('style', 'http://www.wikidata.org/entity/Q529819'), ('styleName', 'Queen Anne style architecture'), ('numBuildings', '147')]
[('style', 'http://www.wikidata.org/entity/Q615196'), ('styleName', 'Italianate architecture'), ('numBuildings', '120')]
[('style', 'http://www.wikidata.org/entity/Q1125300'), ('styleName', 'Georgian architecture'), ('numBuildings', '116')]
[('style', 'http://www.wikidata.org/entity/Q1400086'), ('styleName', 'Federal architecture'), ('numBuildings', '97')]
[('style', 'http://www.wikidata.org/entity/Q2657199'), ('styleName', 'rustic architecture'), ('numBuildings', '92')]
[('style', '

10

I show also the most common types of ***building (wd:Q41176)*** in the ***United States of America (wd:Q30)***.

In [24]:
queryString = """   
SELECT ?type ?typeName COUNT(DISTINCT ?building) AS ?numBuildings WHERE { 
    
    # Retrieve buildings in U.S and the type
    ?building   wdt:P31/wdt:279* wd:Q41176 ;
                wdt:P17          wd:Q30    ;
                wdt:P31          ?type     .

    # I'm only interested in classes differant from BUILDING
    FILTER (?type != wd:Q41176)  
    
    # This returns the labels
    ?type <http://schema.org/name> ?typeName .
}
GROUP BY ?type ?typeName
ORDER BY DESC(?numBuildings)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('type', 'http://www.wikidata.org/entity/Q33506'), ('typeName', 'museum'), ('numBuildings', '57')]
[('type', 'http://www.wikidata.org/entity/Q27686'), ('typeName', 'hotel'), ('numBuildings', '26')]
[('type', 'http://www.wikidata.org/entity/Q839954'), ('typeName', 'archaeological site'), ('numBuildings', '17')]
[('type', 'http://www.wikidata.org/entity/Q123705'), ('typeName', 'neighborhood'), ('numBuildings', '10')]
[('type', 'http://www.wikidata.org/entity/Q163740'), ('typeName', 'nonprofit organization'), ('numBuildings', '9')]
[('type', 'http://www.wikidata.org/entity/Q41253'), ('typeName', 'movie theater'), ('numBuildings', '8')]
[('type', 'http://www.wikidata.org/entity/Q43229'), ('typeName', 'organization'), ('numBuildings', '8')]
[('type', 'http://www.wikidata.org/entity/Q13402009'), ('typeName', 'apartment building'), ('numBuildings', '8')]
[('type', 'http://www.wikidata.org/entity/Q19844914'), ('typeName', 'university building'), ('numBuildings', '8')]
[('type', 'http:

10

#### END TASK 4.1:
I was able to obtain good results with architectural style in the ***United States of America (wd:Q30)***.

For what concern the types of buildings, some of the results don't seem to be a type of building ( like ***nonprofit organization (wd:Q163740)*** ).

This is probably due to the fact that in Wikidata the properties ***instanceOf (wdt:P31)*** and ***subclassOf (wdt:P279)*** are not often consistently used. 

However, I was able to obtain some good results with actual types of building such as museums,hotels, etc...

### ***Task 4.2 :*** In which U.S. states there are more notable buildings?

The property ***country (wdt:P17)*** can not be used since it returns ***United States of America (wd:Q30)*** and not to a specific U.S state.

Maybe, I have to rely on ***located in the administrative territorial entity (wdt:P131)***.

In [25]:
queryString = """
SELECT DISTINCT ?buildingName ?location ?locationName WHERE { 

    # I retrieve only the notable works of architects
    ?architect  wdt:P106 wd:Q42973 ;
                wdt:P800 ?building .

    # Retrieve buildings in U.S
    ?building   wdt:P31/wdt:279* wd:Q41176 ;
                wdt:P17          wd:Q30    ;
                wdt:P131         ?location .

    # This returns the labels
    ?building <http://schema.org/name> ?buildingName .
    ?location <http://schema.org/name> ?locationName .         
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('buildingName', "Washington Governor's Mansion"), ('location', 'http://www.wikidata.org/entity/Q1223'), ('locationName', 'Washington')]
[('buildingName', 'House of Sweden'), ('location', 'http://www.wikidata.org/entity/Q61'), ('locationName', 'Washington, D.C.')]
[('buildingName', 'Palestra'), ('location', 'http://www.wikidata.org/entity/Q1400'), ('locationName', 'Pennsylvania')]
[('buildingName', '747 Wing House'), ('location', 'http://www.wikidata.org/entity/Q99'), ('locationName', 'California')]
[('buildingName', 'Old Washington County Library'), ('location', 'http://www.wikidata.org/entity/Q755559'), ('locationName', 'Hagerstown')]
[('buildingName', 'Athenaeum of Philadelphia'), ('location', 'http://www.wikidata.org/entity/Q1345'), ('locationName', 'Philadelphia')]
[('buildingName', 'Casa De Muchas Flores'), ('location', 'http://www.wikidata.org/entity/Q812'), ('locationName', 'Florida')]
[('buildingName', 'Cascade Theatre'), ('location', 'http://www.wikidata.org/entity/Q

20

I obtained some U.S states: ***Washington, California,Pennyslvania***,etc... 

Since I have also some cities (***Washington D.C***, ***Philadelphia***, etc...), I need to find how to connect a city to a U.S state.

I use ***instanceOf (wdt:P31)*** on ***Washington (wd:Q1223)***.

In [26]:
queryString = """
SELECT ?class ?className WHERE { 

    # Retrieve class of Washington
    wd:Q1223  wdt:P31 ?class.

    # This returns the labels
    ?class <http://schema.org/name> ?className .
}
"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q35657'), ('className', 'U.S. state')]


1

I retrieved ***U.S. state (wdt:Q35657)***.

Now I can use ***city of the United States (wd:Q1093829)*** discovered in one of the first queries to see how cities are connected to ***U.S. state (wdt:Q35657)***.

In [27]:
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 

    # Retrieve cities
    ?city wdt:P31 wd:Q1093829 .
    
    # Retrieve U.S. States
    ?state wdt:P31 wd:Q35657 .
    
    # Connect cities and states
    ?city ?p ?state .

    # This returns the labels
    ?p <http://schema.org/name> ?pName .
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1376'), ('pName', 'capital of')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P47'), ('pName', 'shares border with')]


5

So to connect a city to a ***U.S. state (wdt:Q35657)***, I can probably use the properties ***located in the administrative territorial entity (wdt:P131)*** and ***partOf (wdt:P361)***.

I show some examples of couples [city, U.S State]

In [28]:
queryString = """
SELECT DISTINCT ?cityName ?stateName WHERE { 

    # Retrieve cities
    ?city   wdt:P31              wd:Q1093829 ;
           (wdt:P131 | wdt:P361) ?state      .
    
    # Retrieve U.S. States
    ?state wdt:P31 wd:Q35657 .
    
    # This returns the labels
    ?city  <http://schema.org/name> ?cityName  .
    ?state <http://schema.org/name> ?stateName .
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('cityName', 'Montgomery'), ('stateName', 'West Virginia')]
[('cityName', 'Smithers'), ('stateName', 'West Virginia')]
[('cityName', 'Holland'), ('stateName', 'Michigan')]
[('cityName', 'Grosse Pointe Shores'), ('stateName', 'Michigan')]
[('cityName', 'South Haven'), ('stateName', 'Michigan')]
[('cityName', 'Flat Rock'), ('stateName', 'Michigan')]
[('cityName', 'Milan'), ('stateName', 'Michigan')]
[('cityName', 'Memphis'), ('stateName', 'Michigan')]
[('cityName', 'Paden City'), ('stateName', 'West Virginia')]
[('cityName', 'Fenton'), ('stateName', 'Michigan')]


10

I can associate a ***U.S. state(wd:Q35657)*** to each ***building (wd:Q41176)*** using this BGP:
```
wdt:P131/(wdt:P131 | wdt:P361)* ?state .
```
Finally, I can count the number of buildings for each ***U.S. state(wd:Q35657)***

In [29]:
queryString = """
SELECT ?state ?stateName COUNT(DISTINCT ?building) AS ?numBuildings WHERE { 

    # I retrieve only the notable works of architects
    ?architect  wdt:P106 wd:Q42973 ;
                wdt:P800 ?building .

    # Retrieve buildings in U.S and their state
    ?building   wdt:P31/wdt:279*                wd:Q41176 ;
                wdt:P17                         wd:Q30    ;
                wdt:P131/(wdt:P131 | wdt:P361)* ?state    .

    # I want only U.S states
    ?state wdt:P31 wd:Q35657 .

    # This returns the labels
    ?building <http://schema.org/name> ?buildingName .
    ?state    <http://schema.org/name> ?stateName    .           
}
GROUP BY ?state ?stateName
ORDER BY DESC(?numBuildings)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('state', 'http://www.wikidata.org/entity/Q1204'), ('stateName', 'Illinois'), ('numBuildings', '10')]


1

#### END TASK 4.2:
I discovered that ***Illinois (wd:Q1204)*** is the ***U.S. state(wd:Q35657)*** with the highest number of notable works.

### ***Task 4.3 :*** Which U.S. state contains the largest number of building designed by U.S. architects

The query is similar to the one used in Task 4.2 but:
* I have to add also a path for the nationality of the architects
* I have to use ***architect(wdt:P84)*** instead of **notable work (wdt:P800)***.

In [30]:
queryString = """
SELECT ?state ?stateName COUNT(DISTINCT ?building) AS ?numBuildings WHERE { 

    # I retrieve only american architects
    ?architect  wdt:P106 wd:Q42973 ;
                wdt:P27  wd:Q30    .

    # Retrieve buildings in U.S and their state
    ?building   wdt:P31/wdt:279*                wd:Q41176 ;
                wdt:P84                         ?architect ;
                wdt:P17                         wd:Q30    ;
                wdt:P131/(wdt:P131 | wdt:P361)* ?state    .


    # I want only U.S states
    ?state wdt:P31 wd:Q35657 .

    # This returns the labels
    ?building <http://schema.org/name> ?buildingName .
    ?state    <http://schema.org/name> ?stateName    .            
}
GROUP BY ?state ?stateName
ORDER BY DESC(?numBuildings)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('state', 'http://www.wikidata.org/entity/Q1384'), ('stateName', 'New York'), ('numBuildings', '60')]


1

#### END TASK 4.2:
I discovered that ***New York (wd:Q1384')*** is the ***U.S. state(wd:Q35657)*** with the highest number of buildings designe by U.S. architects.

### Extra analytics query

#### How many buildings from american architects are present in different countries ?

In [31]:
queryString = """
SELECT ?country ?countryName COUNT(DISTINCT ?building) AS ?numBuildings WHERE { 
        
   # I retrieve only american architects
   ?architect  wdt:P106 wd:Q42973 ;
               wdt:P27 wd:Q30 .

   # Retrieve buildings, their country and their architects
   ?building   wdt:P31/wdt:279* wd:Q41176 ;
               wdt:P84          ?architect ;
               wdt:P17          ?country  .
            
    # This returns the labels
    ?country <http://schema.org/name> ?countryName .
}
GROUP BY ?country ?countryName
ORDER BY DESC(?numBuildings)
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America'), ('numBuildings', '529')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('countryName', 'Finland'), ('numBuildings', '11')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('numBuildings', '6')]
[('country', 'http://www.wikidata.org/entity/Q213'), ('countryName', 'Czech Republic'), ('numBuildings', '6')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('countryName', "People's Republic of China"), ('numBuildings', '5')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('countryName', 'Canada'), ('numBuildings', '5')]
[('country', 'http://www.wikidata.org/entity/Q212'), ('countryName', 'Ukraine'), ('numBuildings', '2')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('numBuildings', '2')]
[('country', 'http://www.wikidata.org/entity/Q668'), ('countryName', 'India'), ('numBuildings', '2')]
[('country', 'htt

26

### On average how many buildings from american architects are present in other countries without considering U.S ?

In [32]:
queryString = """
SELECT AVG(?numBuildings) AS ?avgNumBuildingOutsideUS WHERE {

     {     SELECT ?country ?countryName COUNT(DISTINCT ?building) AS ?numBuildings WHERE { 

               # I retrieve only american architects
               ?architect  wdt:P106 wd:Q42973 ;
                           wdt:P27 wd:Q30 .

               # Retrieve buildings, their country and their architects
               ?building   wdt:P31/wdt:279* wd:Q41176 ;
                           wdt:P84          ?architect ;
                           wdt:P17          ?country  .

                # I want to exclude buildings in the U.S
                FILTER (?country != wd:Q30)        

                # This returns the labels
                ?country <http://schema.org/name> ?countryName .
            }
            GROUP BY ?country ?countryName
    }
}
"""

print("Results")
run_query(queryString)

Results
[('avgNumBuildingOutsideUS', '2.52')]


1

### Show the top 10 architectural style most used by american architects in their notable works ?

In [33]:
queryString = """
SELECT ?style ?styleName COUNT(DISTINCT ?building) AS ?numBuildings WHERE { 

    # I retrieve only american architects and their notableWorks
    ?architect  wdt:P106 wd:Q42973 ;
                wdt:P27  wd:Q30    .

    # Retrieve buildings, their country and their architects
    ?building   wdt:P31/wdt:279* wd:Q41176 ;
                wdt:P84          ?architect ;
                wdt:P149         ?style .

    # This returns the labels
    ?style <http://schema.org/name> ?styleName .
}
GROUP BY ?style ?styleName 
ORDER BY DESC(?numBuildings)
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('style', 'http://www.wikidata.org/entity/Q54111'), ('styleName', 'Neoclassical architecture'), ('numBuildings', '27')]
[('style', 'http://www.wikidata.org/entity/Q502163'), ('styleName', 'Renaissance Revival architecture'), ('numBuildings', '20')]
[('style', 'http://www.wikidata.org/entity/Q173782'), ('styleName', 'Art Deco'), ('numBuildings', '16')]
[('style', 'http://www.wikidata.org/entity/Q245188'), ('styleName', 'modern architecture'), ('numBuildings', '14')]
[('style', 'http://www.wikidata.org/entity/Q1268134'), ('styleName', 'Beaux-Arts'), ('numBuildings', '14')]
[('style', 'http://www.wikidata.org/entity/Q3645460'), ('styleName', 'Spanish Colonial Revival architecture'), ('numBuildings', '11')]
[('style', 'http://www.wikidata.org/entity/Q1125300'), ('styleName', 'Georgian architecture'), ('numBuildings', '9')]
[('style', 'http://www.wikidata.org/entity/Q34636'), ('styleName', 'Art Nouveau'), ('numBuildings', '8')]
[('style', 'http://www.wikidata.org/entity/Q744373'), 

10

### On average how many styles are used for each building designed by an american architect ?

In [34]:
queryString = """
SELECT AVG(?numStyles) AS ?avgNumStyles WHERE{

    {    SELECT ?building COUNT(DISTINCT ?style) AS ?numStyles WHERE { 

            # I retrieve only american architects and their notableWorks
            ?architect  wdt:P106 wd:Q42973 ;
                        wdt:P27  wd:Q30    .

            # Retrieve buildings, their country and their architects
            ?building   wdt:P31/wdt:279* wd:Q41176 ;
                        wdt:P84          ?architect ;
                        wdt:P149         ?style .
        }
        GROUP BY ?building
    }
}


"""

print("Results")
run_query(queryString)

Results
[('avgNumStyles', '1.065384615384615')]


1

## END